In [1]:
import numpy as np
import pandas as pd
import dask.dataframe as dd

In [6]:
df = pd.read_csv('C:\\Users\\Aruna\\Documents\\ACMS-IID\\data\\cleandata.csv')
df.head()

,id,label,description
0,1737.0,Amazon SNS,sms failure delivery retrieval detailed sms lo...
1,1736.0,Amazon SNS,messages coming multiple accounts single sms s...
2,1735.0,Amazon SNS,android fcm support unity plugin unity plug-in...
3,1734.0,Amazon SNS,otp message delivery issue facing issue otp me...
4,1733.0,Amazon SNS,simulating message persistence using sqs evalu...


In [7]:
df['label'].value_counts()

Amazon EC2                  295113
Amazon S3                    53443
Amazon RDS                   39107
Amazon Elastic Beanstalk     22345
Amazon VPC                   21629
Amazon CloudFront            19189
Amazon Lambda                 8946
Amazon SNS                    4601
Name: label, dtype: int64

In [13]:
df['description'] = df['description'].apply(lambda x: " ".join(x for x in str(x).split()))

In [14]:
from sklearn.model_selection import train_test_split

X = df['description']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from dask.distributed import Client 
client = Client() # start a local Dask client

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer




nb = Pipeline([('vect', TfidfVectorizer()), ('clf', MultinomialNB())])
nb.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...rue,
        vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [ ]:
TfidfVectorizer?

In [ ]:
'''from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 0.1, stop_words='english', max_features=100)

X_train_tfidf = vectorizer.fit_transform(X_train.values.astype('str')) # remember to use the original X_train set
X_train_tfidf.shape'''

In [ ]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

In [ ]:
from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('rfc',LinearSVC()),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train)  

In [17]:
predictions = nb.predict(X_test)

In [18]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[   92  5375     0     0     0   308     0     0]
 [    1 88551     3     0     9    29     0     1]
 [    0  6514   113     0     5     5     0     0]
 [    0  2673     1     5     1    35     0     0]
 [    0 11064     0     0   691     0     0     0]
 [    1 13464     0     0     0  2542     0     0]
 [    0  1420     0     0     0     5     1     0]
 [    0  6372     0     0     0     2     0    29]]


In [19]:
print(metrics.classification_report(y_test,predictions))

                          precision    recall  f1-score   support

       Amazon CloudFront       0.98      0.02      0.03      5775
              Amazon EC2       0.65      1.00      0.79     88594
Amazon Elastic Beanstalk       0.97      0.02      0.03      6637
           Amazon Lambda       1.00      0.00      0.00      2715
              Amazon RDS       0.98      0.06      0.11     11755
               Amazon S3       0.87      0.16      0.27     16007
              Amazon SNS       1.00      0.00      0.00      1426
              Amazon VPC       0.97      0.00      0.01      6403

               micro avg       0.66      0.66      0.66    139312
               macro avg       0.93      0.16      0.16    139312
            weighted avg       0.76      0.66      0.55    139312



In [20]:
print(metrics.accuracy_score(y_test,predictions))

0.660560468588492
